In [1]:
import cv2
from cnn import CNN
import tensorflow as tf
cnn = CNN()

In [2]:
data_list = [
    {"folder":"img_face","label":"0"},
    {"folder":"img_face_non_obama","label":"1"}
]
    
    
test_list = [
    {"folder":"img_face_test", "label":"0"},
    {"folder":"img_face_test_non_obama", "label":"1"}
]

train_image, train_label, _ = cnn.read_image_cv(data_list)
test_image , test_label , test_image_list = cnn.read_image_cv(test_list)  


print(len(train_image))
print(len(test_image))

220
5


In [5]:
with tf.Graph().as_default():
        
    images_placeholder = tf.placeholder("float", shape=(None, cnn.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float", shape=(None, cnn.NUM_CLASSES))
    keep_prob = tf.placeholder("float")

    
    softmax = cnn.inference(images_placeholder, keep_prob)
    loss_value = cnn.loss(softmax, labels_placeholder)
    train_op = cnn.training(loss_value, cnn.LEARNING_RATE)
    acc = cnn.accuracy(softmax, labels_placeholder)
    
    saver = tf.train.Saver()

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
            
    #summary_op = tf.summary.merge_all()
    #summary_writer = tf.summary.FileWriter (cnn.LOG_DIR, sess.graph)
        
        
    for step in range(cnn.MAX_STEPS):
        for i in range(len(train_image)/cnn.BATCH_SIZE):
            batch = cnn.BATCH_SIZE*i
            sess.run(train_op, feed_dict={
                images_placeholder: train_image[batch:batch+cnn.BATCH_SIZE],
                labels_placeholder: train_label[batch:batch+cnn.BATCH_SIZE],
            keep_prob: 0.5})

            #train_accuracy = sess.run(acc, feed_dict={
            #    images_placeholder: train_image,
            #    labels_placeholder: train_label,
            #    keep_prob: 1.0})
            #print "step %d, training accuracy %g"%(step, train_accuracy)
               
        train_loss = sess.run(loss_value,  feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print "step %d, xentropy %s"%(step, str(train_loss))
            
        if step % 10 == 0 and step > 0 and train_loss < 5.0:
            cnn.display_test_prob(sess, softmax, test_image, "obama", images_placeholder,keep_prob)

            #summary_str = sess.run(summary_op, feed_dict={
            #    images_placeholder: train_image,
            #    labels_placeholder: train_label,
            #    keep_prob: 1.0})
            #summary_writer.add_summary(summary_str, step)

  
    print "test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
       keep_prob: 1.0})
    

    save_path = saver.save(sess, "model.ckpt")

step 0, xentropy 2624.0
step 1, xentropy 1484.24
step 2, xentropy 190.571
step 3, xentropy 165.152
step 4, xentropy 98.7756
step 5, xentropy 133.121
step 6, xentropy 111.83
step 7, xentropy 74.2802
step 8, xentropy 76.3706
step 9, xentropy 53.936
step 10, xentropy 57.5725
step 11, xentropy 45.3017
step 12, xentropy 45.2449
step 13, xentropy 32.2387
step 14, xentropy 40.7304
step 15, xentropy 28.8294
step 16, xentropy 25.959
step 17, xentropy 27.3314


KeyboardInterrupt: 